In [155]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [156]:
import sys
import os

# Add the src directory to the Python path
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

In [157]:
from src.activation_functions import *
from src.loss_functions import *
from src.random_search import *
from src.batch_normalization import *
from src.layer import *
from src.early_stopping import *
from src.utils import *
from src.optimizers import *
import itertools
import pandas as pd
import numpy as np
import sys
import os


In [158]:
df_path = "../ML_project/data/cup/ML-CUP24-TR.csv"
df_path2 = "../ML_project/data/cup/ML-CUP24-TS.csv"

# Preview the dataset
df = pd.read_csv(df_path, skiprows=7, header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1,0.946373,0.307995,0.820058,-0.309386,0.950936,-0.000913,-0.093048,-0.029318,0.995230,-1.175176,2.295016,0.223732,-0.011599,-0.503652,-5.564158
1,2,-0.074075,-0.992654,-1.162582,0.997240,-0.074209,-0.002159,-0.004956,-0.095561,0.995411,0.675930,3.147029,-0.297508,-0.361982,-0.186246,3.445744
2,3,0.841384,0.531605,0.810176,-0.534053,0.845451,-0.001057,-0.082832,-0.051079,0.995254,-1.069958,2.415989,0.234664,0.170027,-0.344143,-3.675575
3,4,0.737117,-0.668400,-1.065601,0.671734,0.740793,0.000008,0.073704,-0.066844,0.995038,11.038418,1.003455,-0.100332,-1.032355,-1.184874,15.554511
4,5,-0.987858,-0.119426,-1.072857,0.120013,-0.992772,0.000064,-0.098661,-0.011863,0.995050,14.834930,1.127759,-0.112600,-0.134817,2.047468,20.522760


In [159]:
df.columns = ["ID"] + [f'input_{i}' for i in range(12)] + ['target_x', 'target_y', 'target_z']
df = df.drop("ID", axis=1)

In [160]:
X_train, X_val, y_train, y_val, X_scaler, y_scaler = preprocess_data(
    df, 
    target=["target_x", "target_y", "target_z"],
    normalize_type="z-score",
    val_ratio=0.2,
    regression=True
)

           0         1          2
0  -0.421997  0.236717  -7.342594
1  -0.283333  0.050838   3.428940
2   0.137419  0.030498  -2.907336
3   1.758643 -0.886580  16.933458
4  -1.091619  0.158737   2.392301
5  -0.030139 -0.041892  -0.703612
6  -0.537285  0.111040   2.657905
7  -0.296103  0.080915   2.885263
8  -1.578751 -0.185479  14.876680
9   1.155768 -0.025354  12.329478
10  0.237865  0.755455   7.693303
11  0.880421  0.209778  -8.828166
12  0.687578  0.985988  14.896379
13  0.762072  0.217130  -9.108539
14 -2.156456 -0.696712  20.812513
15 -0.494655  0.115973  -5.513408
16 -1.620315 -0.362419  17.202943
17  0.149752  0.676320   8.854500
18 -0.794149 -1.553299  17.377104
19 -0.049514  0.745878   8.182392
20  2.333131 -0.558883  24.923707
21  1.227097  0.381899  14.242438
22 -0.740300  0.539295   8.972121
23 -1.312315 -1.067413  16.626081
24  0.588437  0.455289  -9.114358
25 -1.013359 -0.028985   8.870993
26  1.082475 -1.512010  18.644240
27  0.109942  0.027452  -2.324193
28  0.602006  

In [161]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((200, 12), (200, 3), (50, 12), (50, 3))

In [162]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

In [163]:
param_distributions = {
    'hidden_size': [3, 4, 5, 6],
    'n_h_layers': [3, 4, 5],
    'hidden_activation': [Activation_Tanh, Activation_Leaky_ReLU, Activation_Sigmoid, Activation_ReLU],
    'batch_norm': [False],
    'learning_rate': np.logspace(-3, -1, num=20).tolist(),
    'l1':  np.logspace(-5, -1, num=10).tolist(),
    'l2': np.logspace(-5, -1, num=10).tolist(),
    'dropout_rate': np.logspace(-5, -1, num=10).tolist(),
    'batch_size': [8, 16, 32],
    'n_epochs': [50,100],
    'weight_decay': np.logspace(-5, -1, num=20).tolist(),
    'patience': [10, 30, 50],
    'CC': [False],
    'weights_init': ['gaussian', 'gaussian_scaled', 'xavier', 'he', 'random'],

}

In [164]:
best_hyperparams, best_performance = random_search(X_train=X_train,
                                                   y_train=y_train,
                                                   param_distributions=param_distributions,
                                                   n_iters=50, regression=True)  # adjust n_iters as needed

print(best_hyperparams)

3 <class 'src.activation_functions.Activation_ReLU'> 1e-05 False
Data shapes:
X_train: (160, 12), y_train: (160, 3)
Hyperparams: {'hidden_size': 3, 'n_h_layers': 4, 'hidden_activation': <class 'src.activation_functions.Activation_ReLU'>, 'batch_norm': False, 'learning_rate': 0.001, 'l1': 0.03593813663804626, 'l2': 0.00021544346900318823, 'dropout_rate': 1e-05, 'batch_size': 32, 'n_epochs': 100, 'weight_decay': 0.03792690190732246, 'patience': 30, 'CC': False, 'weights_init': 'random'}
Epoch 0: Train Loss: 1.2304, Acc: -24.06% | Val Loss: 1.2977, Acc: -33.47%
Epoch 10: Train Loss: 1.2186, Acc: -22.57% | Val Loss: 1.2845, Acc: -32.10%
Epoch 20: Train Loss: 1.2130, Acc: -23.18% | Val Loss: 1.2780, Acc: -31.44%
Epoch 30: Train Loss: 1.2093, Acc: -25.63% | Val Loss: 1.2737, Acc: -31.00%
Epoch 40: Train Loss: 1.2066, Acc: -22.93% | Val Loss: 1.2705, Acc: -30.67%
Epoch 50: Train Loss: 1.2044, Acc: -23.31% | Val Loss: 1.2679, Acc: -30.40%
Epoch 60: Train Loss: 1.2026, Acc: -23.09% | Val Loss: 

d:\SCHOOL STUFF\MASTER'S - COMPUTER SCIENCE - AI\SEM 1\PROJECTS\machine learning\ML_project\src\activation_functions.py:45: RuntimeWarning: overflow encountered in exp
  self.output = 1 / (1 + np.exp(-inputs))


Epoch 40: Train Loss: 0.7734, Acc: 20.62% | Val Loss: 0.7912, Acc: 18.63%
Final Validation score: 0.1859
✅ Fold 1/5 | Validation Accuracy: 0.1859
3 <class 'src.activation_functions.Activation_Leaky_ReLU'> 1e-05 False
Data shapes:
X_train: (160, 12), y_train: (160, 3)
Hyperparams: {'hidden_size': 3, 'n_h_layers': 4, 'hidden_activation': <class 'src.activation_functions.Activation_Leaky_ReLU'>, 'batch_norm': False, 'learning_rate': 0.1, 'l1': 7.742636826811278e-05, 'l2': 7.742636826811278e-05, 'dropout_rate': 1e-05, 'batch_size': 16, 'n_epochs': 50, 'weight_decay': 0.00206913808111479, 'patience': 10, 'CC': False, 'weights_init': 'random'}
Epoch 0: Train Loss: 1.0758, Acc: -15.28% | Val Loss: 1.1074, Acc: 0.58%
Epoch 10: Train Loss: 0.7410, Acc: 20.77% | Val Loss: 0.8674, Acc: 22.12%
Epoch 20: Train Loss: 0.7317, Acc: 21.07% | Val Loss: 0.8668, Acc: 22.18%
Early stopping at epoch 28
Restoring model weights from epoch 18
Final Validation score: 0.1645
✅ Fold 2/5 | Validation Accuracy: 0.1

In [165]:
print(best_hyperparams)

{'hidden_size': 4, 'n_h_layers': 3, 'hidden_activation': <class 'src.activation_functions.Activation_Tanh'>, 'batch_norm': False, 'learning_rate': 0.023357214690901212, 'l1': 2.782559402207126e-05, 'l2': 0.004641588833612777, 'dropout_rate': 2.782559402207126e-05, 'batch_size': 16, 'n_epochs': 100, 'weight_decay': 0.0007847599703514606, 'patience': 10, 'CC': False, 'weights_init': 'xavier', 'val_accuracy': np.float64(0.3840233435172176)}


In [166]:
if best_hyperparams['CC']:
    model = CascadeCorrelation(input_size = 12, output_size=3, activation=Activation_Leaky_ReLU, output_activation = Activation_Sigmoid)
else:
    model = NN(
        l1=best_hyperparams['l1'],
        l2=best_hyperparams['l2'],
        input_size=12,
        hidden_size=best_hyperparams['hidden_size'],
        output_size=3,
        hidden_activation=best_hyperparams['hidden_activation'],
        dropout_rate=best_hyperparams['dropout_rate'],
        use_batch_norm=best_hyperparams['batch_norm'],
        n_h_layers=best_hyperparams['n_h_layers']
    )

train = Train(best_hyperparams, model, regression=True)
train.train_and_evaluate(X_train, y_train, X_val, y_val)
train.test(X_val, y_val)
print(f"Final Validation R² Score: {train.test_score:.4f}")
train.plot(score=True)

4 <class 'src.activation_functions.Activation_Tanh'> 2.782559402207126e-05 False
Data shapes:
X_train: (200, 12), y_train: (200, 3)
Hyperparams: {'hidden_size': 4, 'n_h_layers': 3, 'hidden_activation': <class 'src.activation_functions.Activation_Tanh'>, 'batch_norm': False, 'learning_rate': 0.023357214690901212, 'l1': 2.782559402207126e-05, 'l2': 0.004641588833612777, 'dropout_rate': 2.782559402207126e-05, 'batch_size': 16, 'n_epochs': 100, 'weight_decay': 0.0007847599703514606, 'patience': 10, 'CC': False, 'weights_init': 'xavier', 'val_accuracy': np.float64(0.3840233435172176)}
Epoch 0: Train Loss: 1.1558, Acc: -17.56% | Val Loss: 0.9897, Acc: 2.99%
Epoch 10: Train Loss: 0.6825, Acc: 30.27% | Val Loss: 0.7155, Acc: 29.86%
Epoch 20: Train Loss: 0.6342, Acc: 34.94% | Val Loss: 0.6651, Acc: 34.81%
Epoch 30: Train Loss: 0.6147, Acc: 38.75% | Val Loss: 0.6619, Acc: 35.12%
Epoch 40: Train Loss: 0.5895, Acc: 38.01% | Val Loss: 0.6613, Acc: 35.18%
Early stopping at epoch 44
Restoring model w

ValueError: operands could not be broadcast together with shapes (50,3) (1,4) 